In [51]:
from fastai import *
from fastai.tabular import *

In [52]:
#%matplotlib notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pycountry_convert as pc

In [65]:
df=pd.read_excel('https://github.com/EnergizedTechLabsDo/energized/raw/master/All_Data_final.xlsx')

In [27]:
#df.set_index("Entity")

In [54]:
df.head()

,Entity,Year,GDP,Coal consumption per capita,HDI,Oil consumption per capita,Hydro power production per capita,CO2e per capita,Energy use per capita
0,Algeria,1990,9.200000e+10,0.0254,0.577,9160000,0.00046,2.99,861
1,Algeria,1991,9.090000e+10,0.0256,0.581,9070000,0.00097,2.99,889
2,Algeria,1992,9.250000e+10,0.0297,0.587,9110000,0.00064,2.96,889
3,Algeria,1993,9.060000e+10,0.0237,0.591,9080000,0.00112,2.97,872
4,Algeria,1994,8.980000e+10,0.0219,0.595,8670000,0.00051,3.06,824


In [119]:
df["Entity"].drop_duplicates()
#hinzufügen von continent codes
codes=[]

for value in df["Entity"]:
    country_code = pc.country_name_to_country_alpha2(value, cn_name_format="default")
    continent_name = pc.country_alpha2_to_continent_code(country_code)
    codes.append(continent_name)


df["Continent Code"]=codes
df['Continent Code'].unique()
continente=df.groupby(['Continent Code','Year'])['CO2e per capita'].mean()
test=continente.to_frame()
test.reset_index(inplace=True)

In [127]:
dep_var="CO2e per capita"
cat_names=["Entity","Continent Code",'Year']
cont_names=['GDP','Energy use per capita','Coal consumption per capita']#,'HDI','Oil consumption per capita','Hydro power production per capita','CO2e per capita']
procs=[FillMissing,Categorify,Normalize]
path = untar_data(URLs.ADULT_SAMPLE)

In [128]:
df = df.astype({'CO2e per capita':'float'})

In [129]:
test = TabularList.from_df(df.iloc[480:960].copy(), path=path, cat_names=cat_names, cont_names=cont_names)

In [130]:
data = (TabularList.from_df(df, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(480,960)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

In [131]:
data.show_batch(rows=10)

Entity,Continent Code,Year,GDP,Energy use per capita,Coal consumption per capita,target
Algeria,AF,2012,-0.2808,-0.6839,-0.7706,3.48
Turkmenistan,AS,2001,-0.3621,-0.4019,-0.6901,8.34
Saudi Arabia,AS,2004,-0.1468,0.5267,-0.7820,17.1
Slovak Republic,EU,2007,-0.3245,-0.0433,0.3984,6.81
Argentina,SA,1991,-0.2555,-0.6195,-0.7478,3.54
United Kingdom,EU,1997,0.5881,0.0915,0.2979,9.04
Azerbaijan,AS,2010,-0.3410,-0.6686,-0.7829,3.4
Portugal,EU,1991,-0.2798,-0.5399,-0.3063,4.44
Belgium,EU,2003,-0.1543,0.6524,0.1369,10.8
Czech Republic,EU,2000,-0.2909,0.1620,2.5651,12.0


In [132]:
learn = tabular_learner(data, layers=[200,100], metrics=root_mean_squared_error)

In [146]:
learn.fit(5, 1e-3)

epoch,train_loss,valid_loss,root_mean_squared_error,time
0,6.516394,5.371411,2.064167,00:54
1,5.237906,5.944119,2.110373,00:48
2,4.514091,5.889718,2.135516,00:47
3,4.076488,5.859489,2.120158,00:47
4,3.835729,5.955155,2.144763,00:48


In [153]:
row = df.iloc[70]

In [148]:
#learn.lr_find()

In [149]:
#learn.recorder.plot()

In [116]:
#learn.fit_one_cycle(1)

In [154]:
learn.predict(row)

(FloatItem [16.634792], tensor([16.6348]), tensor([16.6348]))

In [152]:
print(df.iloc[70])

Entity                               Australia
Year                                      2010
GDP                                   1.15e+12
Coal consumption per capita               2.23
HDI                                      0.927
Oil consumption per capita            43700000
Hydro power production per capita       0.0481
CO2e per capita                           17.6
Energy use per capita                     5790
Continent Code                              OC
Name: 70, dtype: object
